In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
data = pd.read_csv("jester-data-1.csv", header = None)
d = data.to_latex()
text_file = open("Data_Output.txt", "w")
text_file.write(d)
text_file.close()

In [3]:
train, validate, test = np.split(data, [int(.8*len(data)), int(.9*len(data))])

In [4]:
del train[0]

In [5]:
n_features = 2

user_ratings = train.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))


In [6]:
latent_item_features

array([[ 0.482,  0.357],
       [ 0.636,  0.026],
       [ 0.208,  0.956],
       [ 0.249,  0.483],
       [ 0.026,  0.463],
       [ 0.811,  0.889],
       [ 0.08 ,  0.965],
       [ 0.629,  0.863],
       [ 0.565,  0.9  ],
       [ 0.431,  0.932],
       [ 0.479,  0.35 ],
       [ 0.118,  0.572],
       [ 0.17 ,  0.047],
       [ 0.347,  0.264],
       [ 0.344,  0.981],
       [ 0.207,  0.761],
       [ 0.086,  0.805],
       [ 0.805,  0.111],
       [ 0.105,  0.728],
       [ 0.862,  0.495],
       [ 0.157,  0.715],
       [ 0.756,  0.411],
       [ 0.934,  0.356],
       [ 0.307,  0.965],
       [ 0.003,  0.855],
       [ 0.662,  0.526],
       [ 0.639,  0.1  ],
       [ 0.924,  0.144],
       [ 0.798,  0.195],
       [ 0.523,  0.997],
       [ 0.633,  0.533],
       [ 0.389,  0.605],
       [ 0.186,  0.169],
       [ 0.179,  0.428],
       [ 0.228,  0.848],
       [ 0.27 ,  0.235],
       [ 0.006,  0.14 ],
       [ 0.315,  0.068],
       [ 0.948,  0.52 ],
       [ 0.353,  0.41 ],


In [7]:
latent_user_preferences

array([[ 0.807,  0.248],
       [ 0.207,  0.471],
       [ 0.629,  0.683],
       ..., 
       [ 0.102,  0.652],
       [ 0.195,  0.297],
       [ 0.794,  0.847]])

In [8]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 30):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%1 == 0 ):
            print mse

In [9]:
sgd()

26.0795115458
25.0832717199
24.7512605796
24.5044605914
24.2676227099
24.0292719432
23.7878073492
23.5435983724
23.2973788434
23.0497334498
22.8008312116
22.5502351919
22.2967571422
22.038370346
21.7722215803
21.4948093294
21.2024158734
20.8918747124
20.5616844987
20.2133091496
19.8522459287
19.4882366597
19.1340974342
18.8032165562
18.5065801048
18.250594586
18.0365490964
17.8616121975
17.7205620251
17.6074121046


In [10]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-3.1  , -4.398, -5.272, ..., -2.295, -4.266, -2.015],
       [ 2.859,  3.4  ,  3.715, ...,  2.687,  3.079,  2.864],
       [ 5.215,  5.261,  5.132, ...,  5.722,  4.39 ,  6.667],
       ..., 
       [ 3.327,  3.214,  3.026, ...,  3.774,  2.616,  4.472],
       [ 1.156,  0.848,  0.582, ...,  1.545,  0.559,  1.966],
       [ 2.119,  2.092,  2.005, ...,  2.364,  1.724,  2.779]])

In [11]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
del data[0]
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,(-7.820|-3.100),(8.790|-4.398),(-9.660|-5.272),(-8.160|-8.875),(-7.520|-3.087),(-8.500|-1.956),(-9.850|-4.709),(4.170|-3.659),(-8.980|-7.496),(-4.760|-2.908),...,(2.820|0.147),(99.000|-1.451),(99.000|1.193),(99.000|-2.151),(99.000|-1.969),(99.000|-1.080),(-5.630|-0.351),(99.000|-2.295),(99.000|-4.266),(99.000|-2.015)
1,(4.080|2.859),(-0.290|3.400),(6.360|3.715),(4.370|4.657),(-2.380|2.606),(-9.660|3.142),(-0.730|2.774),(-5.340|1.902),(8.880|4.339),(9.220|2.983),...,(2.820|1.918),(-4.950|2.422),(-0.290|1.756),(7.860|2.542),(-0.190|2.404),(-2.140|2.328),(3.060|1.978),(0.340|2.687),(-4.320|3.079),(1.070|2.864)
2,(99.000|5.215),(99.000|5.261),(99.000|5.132),(99.000|3.435),(9.030|4.408),(9.270|7.651),(9.030|2.811),(9.270|1.358),(99.000|4.225),(99.000|5.875),...,(99.000|6.446),(99.000|5.975),(99.000|7.299),(9.080|5.438),(99.000|5.230),(99.000|6.159),(99.000|5.980),(99.000|5.722),(99.000|4.390),(99.000|6.667)
3,(99.000|1.234),(8.350|0.835),(99.000|0.499),(99.000|-1.388),(1.800|0.892),(8.160|2.644),(-2.820|-0.313),(6.210|-0.597),(99.000|-0.606),(1.840|1.578),...,(99.000|2.806),(99.000|2.090),(99.000|3.508),(0.530|1.635),(99.000|1.604),(99.000|2.288),(99.000|2.446),(99.000|1.710),(99.000|0.505),(99.000|2.204)
4,(8.500|3.130),(4.610|2.933),(-4.170|2.688),(-5.390|0.854),(1.360|2.563),(1.600|5.050),(7.040|1.151),(4.610|0.311),(-0.440|1.655),(5.730|3.629),...,(5.190|4.572),(5.580|3.958),(4.270|5.358),(5.190|3.455),(5.730|3.341),(1.550|4.152),(3.110|4.155),(6.550|3.630),(1.800|2.342),(1.600|4.346)
5,(-6.170|-4.868),(-3.540|-5.637),(0.440|-6.061),(-8.500|-7.115),(-7.090|-4.382),(-4.320|-5.659),(-8.690|-4.362),(-0.870|-2.899),(-6.650|-6.795),(-1.800|-5.149),...,(-3.540|-3.741),(-6.890|-4.375),(-0.680|-3.649),(-2.960|-4.457),(-2.180|-4.230),(-3.350|-4.272),(0.050|-3.749),(-9.080|-4.708),(-5.050|-5.045),(-3.450|-5.109)
6,(99.000|3.213),(99.000|3.272),(99.000|3.217),(99.000|2.285),(8.590|2.727),(-9.850|4.649),(7.720|1.807),(8.790|0.907),(99.000|2.726),(99.000|3.605),...,(99.000|3.873),(99.000|3.629),(99.000|4.360),(99.000|3.323),(99.000|3.194),(2.330|3.730),(99.000|3.605),(99.000|3.497),(99.000|2.745),(99.000|4.059)
7,(6.840|-2.736),(3.160|-3.537),(9.170|-4.050),(-6.210|-5.979),(-8.160|-2.598),(-1.700|-2.430),(9.270|-3.332),(1.410|-2.455),(-5.190|-5.263),(-4.420|-2.725),...,(7.230|-0.950),(-1.120|-1.851),(-0.100|-0.449),(-5.680|-2.192),(-3.160|-2.047),(-3.350|-1.654),(2.140|-1.180),(-0.050|-2.326),(1.310|-3.316),(0.000|-2.307)
8,(-3.790|-0.674),(-3.540|-1.353),(-9.420|-1.840),(-6.890|-4.063),(-8.740|-0.817),(-0.290|0.383),(-5.290|-1.972),(-8.930|-1.686),(-7.860|-3.186),(-1.600|-0.450),...,(4.370|1.274),(-0.290|0.340),(4.170|1.986),(-0.290|-0.130),(-0.290|-0.072),(-0.290|0.571),(-0.290|0.923),(-0.290|-0.154),(-3.400|-1.445),(-4.950|0.170)
9,(3.010|4.533),(5.150|4.930),(5.150|5.084),(3.010|4.903),(6.410|3.963),(5.150|5.924),(8.930|3.296),(2.520|1.980),(3.010|5.071),(8.160|4.943),...,(99.000|4.487),(4.470|4.605),(99.000|4.793),(99.000|4.424),(99.000|4.227),(99.000|4.630),(99.000|4.300),(99.000|4.663),(99.000|4.281),(99.000|5.249)


In [12]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -3.09999853081)","(8.79, -4.39795826531)","(-9.66, -5.27233622531)","(-8.16, -8.87515419376)","(-7.52, -3.08743937564)","(-8.5, -1.95642796798)","(-9.85, -4.70916286524)","(4.17, -3.65894813415)","(-8.98, -7.49634704806)","(-4.76, -2.90779023604)",...,"(2.82, 0.147128365745)","(99.0, -1.45058668163)","(99.0, 1.19282830241)","(99.0, -2.1508706573)","(99.0, -1.96853749836)","(99.0, -1.08015760464)","(-5.63, -0.351149607174)","(99.0, -2.29497159178)","(99.0, -4.26589603634)","(99.0, -2.0149702984)"
1,"(4.08, 2.85874525184)","(-0.29, 3.39971173462)","(6.36, 3.71511114782)","(4.37, 4.65687072536)","(-2.38, 2.60598050271)","(-9.66, 3.14210393478)","(-0.73, 2.77440511845)","(-5.34, 1.9019681401)","(8.88, 4.33949683182)","(9.22, 2.98318105173)",...,"(2.82, 1.9183578536)","(-4.95, 2.4224232217)","(-0.29, 1.75563160945)","(7.86, 2.541844901)","(-0.19, 2.40409609248)","(-2.14, 2.32833761841)","(3.06, 1.97754074179)","(0.34, 2.68736809286)","(-4.32, 3.07884513211)","(1.07, 2.86389584157)"
2,"(99.0, 5.21509962271)","(99.0, 5.26093287126)","(99.0, 5.13204355617)","(99.0, 3.43488814734)","(9.03, 4.40800395621)","(9.27, 7.65133727567)","(9.03, 2.81149056154)","(9.27, 1.35771922165)","(99.0, 4.2246517048)","(99.0, 5.87488329111)",...,"(99.0, 6.44618345757)","(99.0, 5.97521740177)","(99.0, 7.29924594164)","(9.08, 5.43798423856)","(99.0, 5.23029649547)","(99.0, 6.158651213)","(99.0, 5.97987749425)","(99.0, 5.72158451175)","(99.0, 4.38999723289)","(99.0, 6.66726674174)"
3,"(99.0, 1.23360065635)","(8.35, 0.835297479307)","(99.0, 0.49860874503)","(99.0, -1.38770307835)","(1.8, 0.892257764882)","(8.16, 2.64436540864)","(-2.82, -0.313112929289)","(6.21, -0.597088495354)","(99.0, -0.605785259441)","(1.84, 1.57783419011)",...,"(99.0, 2.80593749064)","(99.0, 2.08996120516)","(99.0, 3.50768948062)","(0.53, 1.63458149635)","(99.0, 1.6044116885)","(99.0, 2.28773017327)","(99.0, 2.4459520805)","(99.0, 1.70954714803)","(99.0, 0.505112643694)","(99.0, 2.20439504456)"
4,"(8.5, 3.13018552336)","(4.61, 2.93320939471)","(-4.17, 2.68784443789)","(-5.39, 0.854484907364)","(1.36, 2.56321770312)","(1.6, 5.05031320683)","(7.04, 1.15081612126)","(4.61, 0.311107849797)","(-0.44, 1.65502996921)","(5.73, 3.62943923211)",...,"(5.19, 4.57202150452)","(5.58, 3.95762888008)","(4.27, 5.35835965416)","(5.19, 3.45504401011)","(5.73, 3.34078796033)","(1.55, 4.15243197261)","(3.11, 4.15514309529)","(6.55, 3.62958985611)","(1.8, 2.34233138019)","(1.6, 4.34597687079)"
5,"(-6.17, -4.86781300801)","(-3.54, -5.63749051326)","(0.44, -6.06118784864)","(-8.5, -7.11508798189)","(-7.09, -4.38172006537)","(-4.32, -5.65924528847)","(-8.69, -4.36207862224)","(-0.87, -2.89868827726)","(-6.65, -6.79498318683)","(-1.8, -5.14936040731)",...,"(-3.54, -3.74082720497)","(-6.89, -4.37532124654)","(-0.68, -3.64883200067)","(-2.96, -4.45713019604)","(-2.18, -4.229592576)","(-3.35, -4.27226665998)","(0.05, -3.74916747189)","(-9.08, -4.70784199269)","(-5.05, -5.04515645776)","(-3.45, -5.10881296564)"
6,"(99.0, 3.21274708314)","(99.0, 3.27245681513)","(99.0, 3.21661003776)","(99.0, 2.28530820191)","(8.59, 2.72711019961)","(-9.85, 4.64938963403)","(7.72, 1.80725715261)","(8.79, 0.90705716975)","(99.0, 2.72606018269)","(99.0, 3.60472988469)",...,"(99.0, 3.87259910181)","(99.0, 3.62896975622)","(99.0, 4.35966495009)","(99.0, 3.32326399134)","(99.0, 3.19386021297)","(2.33, 3.73009772424)","(99.0, 3.60454006972)","(99.0, 3.4973688059)","(99.0, 2.74547147261)","(99.0, 4.05909466631)"
7,"(6.84, -2.73617174945)","(3.16, -3.53684292372)","(9.17, -4.05043885886)","(-6.21, -5.97851217339)","(-8.16, -2.5982576815)","(-1.7, -2.43038206871)","(9.27, -3.33178395931)","(1.41, -2.45533314904)","(-5.19, -5.26326682361)","(-4.42, -2.72516667552)",...,"(7.23, -0.950278085552)","(-1.12, -1.85075684126)","(-0.1, -0.44883250064)","(-5.68, -2.19205854383)","(-3.16, -2.0471079595)","(-3.35, -1.65396145801)","(2.14, -1.1795648782)","(-0.05, -2.32589245145)","(1.31,

In [13]:
d = comparison_data.to_latex()
text_file = open("comparison_new.txt", "w")
text_file.write(d)
text_file.close()